In [34]:
import pandas as pd

In [35]:
df=pd.read_csv("stroke_analysis.csv")

In [36]:
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke,age_category,Bmi_category,glucose_category,risk_factor,risk_factor1
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1,61+,Obese,High,3,High Risk
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1,61+,Missing,High,2,Medium Risk
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1,61+,Obese,Medium,2,Medium Risk
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1,46-60,Obese,High,2,Medium Risk
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1,61+,Normal,High,2,Medium Risk


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   int32  
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                5110 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
 12  age_category       5110 non-null   object 
 13  Bmi_category       5110 non-null   object 
 14  glucose_category   5110 non-null   object 
 15  risk_factor        5110 non-null   int64  
 16  risk_factor1       5110 

In [44]:
df.isnull().sum()

id                   0
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
age_category         0
Bmi_category         0
glucose_category     0
risk_factor          0
risk_factor1         0
dtype: int64

In [39]:
df["bmi"]=df["bmi"].fillna("0")

In [40]:
df["bmi"]=df["bmi"].astype("float")

In [42]:
df["age"]=df["age"].astype("int")

In [50]:
df.columns.to_list()

['id',
 'gender',
 'age',
 'hypertension',
 'heart_disease',
 'ever_married',
 'work_type',
 'Residence_type',
 'avg_glucose_level',
 'bmi',
 'smoking_status',
 'stroke',
 'age_category',
 'Bmi_category',
 'glucose_category',
 'risk_factor',
 'risk_factor1']

In [84]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder,OrdinalEncoder
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV

In [56]:
x = df[["gender","age","hypertension","heart_disease",
        "work_type","avg_glucose_level","bmi","smoking_status","risk_factor1"]]
y=df["stroke"]

In [57]:
x_num_scaler=StandardScaler()
x_cat_encoder=OneHotEncoder(sparse_output=False)
x_cat_ordinal_encoder=OrdinalEncoder()
y_encoder=LabelEncoder()

In [60]:
binary_x = x[["hypertension", "heart_disease"]].values
scaled_x_num = x_num_scaler.fit_transform(x[["age", "avg_glucose_level", "bmi"]])
encoded_x_cat = x_cat_encoder.fit_transform(x[["work_type", "smoking_status"]])
ordinal_encoded_x_cat = x_cat_ordinal_encoder.fit_transform(x[["gender", "risk_factor1"]])

In [61]:
final_x = np.hstack([scaled_x_num, binary_x, encoded_x_cat, ordinal_encoded_x_cat])

In [62]:
final_x

array([[ 1.0509501 ,  2.70637544,  0.92807976, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.78583466,  2.12155854, -2.91300238, ...,  0.        ,
         0.        ,  2.        ],
       [ 1.62536687, -0.0050283 ,  0.4977946 , ...,  0.        ,
         1.        ,  2.        ],
       ...,
       [-0.36299889, -0.51144264,  0.29839416, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.3439756 ,  1.32825706, -0.22634384, ...,  0.        ,
         1.        ,  1.        ],
       [ 0.03467426, -0.46086746, -0.16337528, ...,  0.        ,
         0.        ,  1.        ]])

In [63]:
x_train,x_test,y_train,y_test=train_test_split(final_x,y,test_size=0.20,random_state=42)

In [85]:
models = {
    "LogisticRegression": LogisticRegression(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "RandomForestClassifier": RandomForestClassifier(),
    "SVC": SVC(probability=True),
    "KNeighborsClassifier": KNeighborsClassifier(),
    "XGBClassifier": XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

In [86]:
for i in models:
    model=models[i]
    model.fit(x_train,y_train)
    y_train_pred=model.predict(x_train)
    y_test_pred=model.predict(x_test)
    print(i,"\n","Train Accuracy:",model.score(x_train,y_train),"\n",
          "Test Accuracy:",model.score(x_test,y_test),"\n")
    print(classification_report(y_test,y_test_pred, zero_division=0))

LogisticRegression 
 Train Accuracy: 0.9537671232876712 
 Test Accuracy: 0.9393346379647749 

              precision    recall  f1-score   support

           0       0.94      1.00      0.97       960
           1       0.00      0.00      0.00        62

    accuracy                           0.94      1022
   macro avg       0.47      0.50      0.48      1022
weighted avg       0.88      0.94      0.91      1022

DecisionTreeClassifier 
 Train Accuracy: 1.0 
 Test Accuracy: 0.9148727984344422 

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       960
           1       0.27      0.24      0.26        62

    accuracy                           0.91      1022
   macro avg       0.61      0.60      0.61      1022
weighted avg       0.91      0.91      0.91      1022

RandomForestClassifier 
 Train Accuracy: 1.0 
 Test Accuracy: 0.9383561643835616 

              precision    recall  f1-score   support

           0       0.94      1.

c:\Users\jothe\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:10:19] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [80]:
x_smote=SMOTE(random_state=42)

In [81]:
x_train_resampled, y_train_resampled = SMOTE().fit_resample(x_train, y_train)

In [83]:
y_train_resampled.value_counts()

stroke
0    3901
1    3901
Name: count, dtype: int64

In [87]:
param_grids = {
    "LogisticRegression": {
        'penalty': ['l1', 'l2'],
        'C': [0.01, 0.1, 1, 10],
        'solver': ['liblinear'],
        'max_iter': [100, 200]
    },
    "DecisionTreeClassifier": {
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'criterion': ['gini', 'entropy']
    },
    "RandomForestClassifier": {
        'n_estimators': [100, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2],
        'bootstrap': [True, False]
    },
    "SVC": {
        'C': [0.1, 1, 10],
        'gamma': [1, 0.1, 0.01],
        'kernel': ['rbf', 'linear']
    },
    "KNeighborsClassifier": {
        'n_neighbors': [3, 5, 7],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    },
    "XGBClassifier": {
        'n_estimators': [100, 200],
        'max_depth': [3, 6],
        'learning_rate': [0.01, 0.1],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }
}

In [89]:
for model_name, model in models.items():
    param_grid = param_grids[model_name]
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, 
                               scoring='accuracy', cv=5, n_jobs=-1, verbose=1)
    grid_search.fit(x_train_resampled, y_train_resampled)
    best_model = grid_search.best_estimator_
    y_test_pred = best_model.predict(x_test)
    print(f"Best parameters for {model_name}: {grid_search.best_params_}")
    print(f"Train Accuracy for {model_name}: {best_model.score(x_train_resampled, y_train_resampled)}")
    print(f"Test Accuracy for {model_name}: {best_model.score(x_test, y_test)}")
    print(classification_report(y_test, y_test_pred, zero_division=0))

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best parameters for LogisticRegression: {'C': 0.01, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'}
Train Accuracy for LogisticRegression: 0.7833888746475263
Test Accuracy for LogisticRegression: 0.7514677103718199
              precision    recall  f1-score   support

           0       0.98      0.75      0.85       960
           1       0.17      0.81      0.28        62

    accuracy                           0.75      1022
   macro avg       0.58      0.78      0.57      1022
weighted avg       0.93      0.75      0.82      1022

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best parameters for DecisionTreeClassifier: {'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2}
Train Accuracy for DecisionTreeClassifier: 1.0
Test Accuracy for DecisionTreeClassifier: 0.8718199608610567
              precision    recall  f1-score   support

           0       0.95     

c:\Users\jothe\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:19:40] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Best parameters for XGBClassifier: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 200, 'subsample': 1.0}
Train Accuracy for XGBClassifier: 0.9871827736477826
Test Accuracy for XGBClassifier: 0.9070450097847358
              precision    recall  f1-score   support

           0       0.94      0.96      0.95       960
           1       0.15      0.11      0.13        62

    accuracy                           0.91      1022
   macro avg       0.55      0.54      0.54      1022
weighted avg       0.90      0.91      0.90      1022



In [90]:
best_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, ...)

In [91]:
import joblib

joblib.dump(best_model, "best_stroke_model.pkl")
print("Best model saved as best_stroke_model.pkl")

Best model saved as best_stroke_model.pkl


In [92]:
joblib.dump(x_num_scaler, "x_num_scaler.pkl")
joblib.dump(x_cat_encoder, "x_cat_encoder.pkl")
joblib.dump(x_cat_ordinal_encoder, "x_cat_ordinal_encoder.pkl")

['x_cat_ordinal_encoder.pkl']